<a href="https://colab.research.google.com/github/OConnorw/BIA_DR/blob/main/Ennis_DR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d amanneo/diabetic-retinopathy-resized-arranged
!unzip -q diabetic-retinopathy-resized-arranged.zip

# Set directories for train and test data
train_dir = 'resized_train_cropped'
test_dir = 'resized_test_cropped'

# Load data using ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical', subset='training')
valid_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical', subset='validation')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')


# Model

In [ ]:

# Create model using pre-trained InceptionV3 base
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import InceptionV3

base_model = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
for layer in base_model.layers:
    layer.trainable = False
x = layers.Flatten()(base_model.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(5, activation='softmax')(x)
model = keras.Model(inputs=base_model.input, outputs=x)



# Optomizer/lost/training

In [ ]:
# Compile and train model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Evaluate model on test dataset
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)


# Test

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)